In [1]:
# Install requirements
%pip install SpeechRecognition
%pip install pyaudio
%pip install openvino
%pip install openvino-genai
%pip install transformers torch librosa numpy


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp311-cp311-macosx_10_9_universal2.whl size=39023 sha256=9e6834340906ff737b66370c7af84112d6043b929fd88cbc46f4d35d6ef89c73
  Stored in directory: /Users/anthonynkyi/Library/Caches/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip instal

In [6]:
# Run to convert and generate whisper into OpenVino
# PS: Delete the whisper-tiny folder before pushing changes onto origin


import logging
import nncf
from cmd_helper import optimum_cli
from pathlib import Path


# Set logging level
nncf.set_log_level(logging.ERROR)


# Define model ID and path
model_id = "openai/whisper-base.en"
model_path = Path(model_id.split("/")[1])


# Convert the model using OpenVINO tools
optimum_cli(model_id, model_path)
print(f"✅ {model_id} model converted and can be found in {model_path}")

**Export command:**

`optimum-cli export openvino --model openai/whisper-base.en whisper-base.en`

Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/Users/anthonynkyi/Documents/cs/COMP0016/reading-star/myenv/lib/python3.11/site-packages/transformers/models/whisper/modeling_whisper.py

✅ openai/whisper-base.en model converted and can be found in whisper-base.en


In [11]:
# live stripping of vocals from background
%pip install torchaudio
%pip install mir_eval
%pip install pydub
%pip install ffmpeg
%pip install ffprobe

import certifi
import os
import torch
import torchaudio

from IPython.display import Audio
from mir_eval import separation
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS
from torchaudio.transforms import Fade
from torchaudio.utils import download_asset

os.environ['SSL_CERT_FILE'] = certifi.where()
bundle = HDEMUCS_HIGH_MUSDB_PLUS
model = bundle.get_model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
sample_rate = bundle.sample_rate
print(f"Sample rate: {sample_rate}")

def separate_sources(
    model,
    mix,
    segment=10.0,
    overlap=0.1,
    device=None,
):
    """
    Apply model to a given mixture. Use fade, and add segments together in order to add model segment by segment.

    Args:
        segment (int): segment length in seconds
        device (torch.device, str, or None): if provided, device on which to
            execute the computation, otherwise `mix.device` is assumed.
            When `device` is different from `mix.device`, only local computations will
            be on `device`, while the entire tracks will be stored on `mix.device`.
    """
    if device is None:
        device = mix.device
    else:
        device = torch.device(device)

    batch, channels, length = mix.shape

    chunk_len = int(sample_rate * segment * (1 + overlap))
    start = 0
    end = chunk_len
    overlap_frames = overlap * sample_rate
    fade = Fade(fade_in_len=0, fade_out_len=int(overlap_frames), fade_shape="linear")

    final = torch.zeros(batch, len(model.sources), channels, length, device=device)

    while start < length - overlap_frames:
        chunk = mix[:, :, start:end]
        with torch.no_grad():
            out = model.forward(chunk)
        out = fade(out)
        final[:, :, :, start:end] += out
        if start == 0:
            fade.fade_in_len = int(overlap_frames)
            start += int(chunk_len - overlap_frames)
        else:
            start += chunk_len
        end += chunk_len
        if end >= length:
            fade.fade_out_len = 0
    return final


def plot_spectrogram(stft, title="Spectrogram"):
    magnitude = stft.abs()
    spectrogram = 20 * torch.log10(magnitude + 1e-8).numpy()
    _, axis = plt.subplots(1, 1)
    axis.imshow(spectrogram, cmap="viridis", vmin=-60, vmax=0, origin="lower", aspect="auto")
    axis.set_title(title)
    plt.tight_layout()



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Users/anthonynkyi/Documents/cs/COMP0016/reading-star/myenv/lib/python3.11/site-packages/torchaudio/pipelines/_source_separation_pipeline.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experi

Sample rate: 44100


In [28]:
# Main code, has to be ran after the previous 2


import os
import numpy as np
import speech_recognition as sr
from datetime import datetime, timedelta
from queue import Queue
from time import sleep
from difflib import SequenceMatcher
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from transformers.utils import logging
import openvino_genai as ov_genai
from notebook_utils import device_widget
import librosa
import torch


# Select device
device = device_widget(default="CPU", exclude=["NPU"])


# Initialize OpenVINO pipeline
ov_pipe = ov_genai.WhisperPipeline(str(model_path), device=device.value)


# Known lyrics and matching function remain the same
lyrics = {
   "Verse 1": [
       "Twinkle, twinkle, little star",
       "How I wonder what you are",
       "Up above the world so high",
       "Like a diamond in the sky",
   ]
}


def find_closest_match(transcription, lyrics):
   best_match = ""
   highest_similarity = 0
   for line in lyrics:
       similarity = SequenceMatcher(None, transcription, line).ratio()
       if similarity > highest_similarity:
           highest_similarity = similarity
           best_match = line
   return best_match, highest_similarity


# Audio recording setup
energy_threshold = 1000
record_timeout = 3.0
phrase_timeout = 3.0
phrase_time = None
data_queue = Queue()
recorder = sr.Recognizer()
recorder.energy_threshold = energy_threshold
recorder.dynamic_energy_threshold = False


source = sr.Microphone(sample_rate=16000)


def record_callback(_, audio: sr.AudioData) -> None:
   data = audio.get_raw_data()
   data_queue.put(data)


# Main processing loop
with source:
   recorder.adjust_for_ambient_noise(source)


stop_call = recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)
print("Model loaded and microphone initialized.\n")

t = np.linspace(0, 1, 16000, endpoint=False)
frequency = 440  # A4 note

try:
   current_verse = "Verse 1"
   while True:
       now = datetime.utcnow()
       if not data_queue.empty():
           if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
               phrase_complete = True
           phrase_time = now

           # Get audio data
           audio_data = b''.join(data_queue.queue)
           data_queue.queue.clear()

           # Convert to numpy array
           audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0
           waveform = 0.5 * np.sin(2 * np.pi * frequency * t)
           normalized_waveform = np.int16(waveform / np.max(np.abs(waveform)) * 32767)

           ref = normalized_waveform.mean(0)
           normalized_waveform = (normalized_waveform - ref.mean()) / ref.std()  # normalization
           normalized_waveform = normalized_waveform.astype(np.float32)

           sources = separate_sources(
               model, 
               mix=normalized_waveform[None], 
               device="cpu",
               segment=1,
               overlap=0.1
            )[0]
           
           sources_list = model.sources
           sources = list(sources)
           audios = dict(zip(sources_list, sources))
           
           # Display audio sources
           vocals = audios["vocals"].cpu()

           # Process with OpenVINO pipeline
           genai_result = ov_pipe.generate(vocals)

           recognized_text = str(genai_result).strip()
           # Match with lyrics
           match, similarity = find_closest_match(recognized_text, lyrics[current_verse])

           # Clear screen and display results
           os.system('clear')
           print(f"\nRecognized: {recognized_text}")
           print(f"Best Match: {match} (Similarity: {similarity:.2f})")
       else:
           sleep(0.1)


except KeyboardInterrupt:
   print("\nTranscription stopped by user.")
   stop_call()

Model loaded and microphone initialized.



/var/folders/h_/lg0zhls57_l_8r2q6kbmsszw0000gn/T/ipykernel_68445/2736584775.py:98: RuntimeWarning: divide by zero encountered in divide
  normalized_waveform = (normalized_waveform - ref.mean()) / ref.std()  # normalization
/var/folders/h_/lg0zhls57_l_8r2q6kbmsszw0000gn/T/ipykernel_68445/2736584775.py:98: RuntimeWarning: invalid value encountered in divide
  normalized_waveform = (normalized_waveform - ref.mean()) / ref.std()  # normalization


ValueError: not enough values to unpack (expected 3, got 2)